# Part 4: Using GPU acceleration with PyTorch

In [4]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try:
    import torchbearer
except:
    !pip install torchbearer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [8]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-1.1208e-02,  9.3118e-01, -8.3674e-01,  ..., -5.8291e-03,
         -1.3640e-01,  3.2634e-02],
        [-3.9581e-02, -1.1751e-03,  7.1772e-01,  ...,  1.4057e-01,
         -1.5655e+00, -2.4921e-02],
        [ 8.8281e-01, -1.8873e+00, -5.6056e-01,  ...,  3.1254e-02,
         -8.6173e-02,  3.6146e+00],
        ...,
        [-6.7565e-01, -1.5333e+00, -7.4142e-01,  ..., -7.3702e-02,
         -1.0272e+00,  3.0715e-01],
        [-4.3137e-01,  6.1237e-02,  4.9141e-01,  ..., -6.9612e-01,
         -1.2813e+00, -6.7493e-02],
        [-6.0365e-01,  5.2043e-01, -2.7698e+00,  ...,  2.6609e-01,
         -3.9514e-02, -1.2385e-01]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [9]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-0.2958, -0.3437,  0.2052,  ...,  0.6735,  0.3735, -0.4756],
        [-1.4573,  1.5404,  0.3775,  ...,  0.0438,  0.0697,  0.2103],
        [ 0.4384,  0.0211, -0.1774,  ..., -0.5405,  0.9608, -0.4510],
        ...,
        [ 0.9367,  2.0611,  0.1589,  ..., -0.6964,  0.8030, -0.0041],
        [-0.3163,  0.9426, -1.1771,  ...,  0.5272,  0.0468, -0.0892],
        [-0.5608, -2.7546,  0.5678,  ...,  0.1039,  0.2439,  1.8849]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-5.1983e-02,  2.5491e-01, -1.1860e-01,  ...,  9.7369e-01,
          1.6014e-02, -1.9318e-01],
        [-1.6515e-02, -9.8078e-01, -1.0485e-01,  ..., -1.3768e+00,
         -1.4036e-01, -1.1407e+00],
        [ 1.2598e+00, -5.1001e-02, -2.1030e-02,  ..., -9.9371e-01,
         -5.8089e-01,  2.4739e-01],
        ...,
        [-5.4655e-01, -1.3942e+00, -8.9394e-01,  ..., -2.3460e-01,
          5.4407e-01, -1.0078e+00],
        [-1.0908e-02,  9.2118e-02, -5.4891e-01,  ..., -4.0917e-01,
          1.4886e+00,  3.9321e-01],
        [ 7.9195e-04,  6.1812e-01, -2.5358e+00,  ...,  5.5795e-03,
         -1.6103e-01,  2.2402e-02]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

Large number of threads.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

The amount of available memory on the GPU(s) and the bandwidth of GPU(s) memory.